# 📦 YOLOv8 Training: Merged Dataset (Products + Boxes)
**Project:** AI Vision Logistics
**Reference:** [Original Colab Notebook](https://colab.research.google.com/drive/1lfOpmo884ovO5S1kSxLPU-2wlNZb5OUC)
**Goal:** Train a model capable of detecting specific products and generic shipping boxes/labels.

### Data Sources
1. **Products Counter** (v2): Specific product SKUs.
2. **Package & Label Detector** (v3): Logistical elements.

---

In [ ]:
# 1. Environment Setup
%pip install ultralytics roboflow -q
from ultralytics import YOLO
from roboflow import Roboflow
import os
import shutil

print("✅ Libraries Installed")

In [ ]:
# 2. Download Dataset A: Products
rf = Roboflow(api_key="9Dro6WbBZ9bW4iaU1Z53")
project = rf.workspace("hamza-dr69s").project("products-couter-yxfxm")
version = project.version(2)
dataset_a = version.download("yolov8")

print(f"✅ Downloaded Primary: {dataset_a.location}")

In [ ]:
# 3. Download Dataset B: Labels & Packages
rf = Roboflow(api_key="9Dro6WbBZ9bW4iaU1Z53")
project = rf.workspace("larry-motalavigne").project("package-and-label-detector")
version = project.version(3)
dataset_b = version.download("yolov8")

print(f"✅ Downloaded Secondary: {dataset_b.location}")

In [ ]:
# 4. MERGE DATASETS
# We copy images and labels from Dataset B into Dataset A to create one Unified Dataset.

def merge_directories(src_dir, dst_dir):
    for split in ['train', 'valid', 'test']:
        for type_ in ['images', 'labels']:
            src = os.path.join(src_dir, split, type_)
            dst = os.path.join(dst_dir, split, type_)
            
            if os.path.exists(src):
                print(f"🔄 Merging {src} -> {dst}...")
                # Copy all files
                for file in os.listdir(src):
                    shutil.copy(os.path.join(src, file), os.path.join(dst, file))

print("🚀 Starting Merge...")
merge_directories(dataset_b.location, dataset_a.location)
print("✅ MEREGE COMPLETE. usage: Unified Dataset (Dataset A)")

In [ ]:
# 5. Train Model (On Unified Data)
model = YOLO("yolov8n.pt")

# dataset_a.location now contains BOTH datasets
model.train(
    data=f"{dataset_a.location}/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name="roboflow_v2_merged"
)

In [ ]:
# 6. Download Trained Model
from google.colab import files

model_path = "/content/runs/detect/roboflow_v2_merged/weights/best.pt"
print(f"⬇️ Downloading Model from: {model_path}")

files.download(model_path)